# **Real-Time Face Mask Detection**

In this task, I bill build a real-time face mask detection to detect whether the person on the webcam is wearing a mask or not.

The dataset use in this task consists of 1376 images with 690 images containing images of people wearing masks and 686 images with people without masks.

In [6]:
!unzip /content/drive/MyDrive/Dataset/test.zip

Archive:  /content/drive/MyDrive/Dataset/test.zip
replace test/with_mask/1-with-mask.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip /content/drive/MyDrive/Dataset/train.zip

Archive:  /content/drive/MyDrive/Dataset/train.zip
   creating: train/
   creating: train/with_mask/
  inflating: train/with_mask/-110603108-gettyimages-533567012.jpg  
  inflating: train/with_mask/-110695084-mask2.jpg  
  inflating: train/with_mask/0-with-mask.jpg  
  inflating: train/with_mask/0000305-62360-halyard-health-pfr-p3-respirator-face-mask-ffp.jpeg  
  inflating: train/with_mask/0000444-48296-halyard-health-surgical-mask-teddy-bear-print.jpeg  
  inflating: train/with_mask/007563.jpg  
  inflating: train/with_mask/0779db20580fc7b054ad6b1b7bd9fb35.jpeg  
  inflating: train/with_mask/1-respirator-mask.jpg  
  inflating: train/with_mask/10-with-mask.jpg  
  inflating: train/with_mask/100-with-mask.jpg  
  inflating: train/with_mask/101-with-mask.jpg  
  inflating: train/with_mask/103-with-mask.jpg  
  inflating: train/with_mask/104-with-mask.jpg  
  inflating: train/with_mask/105-with-mask.jpg  
  inflating: train/with_mask/106.jpg  
  inflating: train/with_mask/107-with-mask.

# 1. **Import libraries:**

1.   **Keras :**used for distributed training of deep learning models
2.   **Sklearn :**provides a selection of efficient tools for machine learning and statistical modeling including classification, regression, clustering and dimensionality reduction via a consistence interface in Python
3.   **Imutils :** A series of convenience functions to make basic image processing functions such as translation, rotation, resizing, skeletonization, displaying Matplotlib images, sorting contours, detecting edges
4.   **Numpy :** offers comprehensive mathematical functions, random number generators, linear algebra routines, Fourier transforms, and more



In [ ]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

# 2.**Build the neural network**

This convolution network consists of two pairs of Conv and MaxPool layers to extract features from the dataset. Which is then followed by a Flatten and Dropout layer to convert the data in 1D and ensure overfitting.

In [7]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# 3. **Image data Generation**

In [8]:
TRAINING_DIR = "/content/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "/content/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


# **4. Initialize a callback checkpoint to keep saving best model after each epoch while training**

In [9]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

# **5.Train the model**

this process take 32 minutes to complete

In [10]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

<ipython-input-10-6272b23e5a0b>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/10
132/132 [==============================] - ETA: 0s - loss: 0.7012 - acc: 0.5270

132/132 [==============================] - 195s 1s/step - loss: 0.7012 - acc: 0.5270 - val_loss: 0.6103 - val_acc: 0.8557
Epoch 2/10
132/132 [==============================] - ETA: 0s - loss: 0.4516 - acc: 0.8091

132/132 [==============================] - 186s 1s/step - loss: 0.4516 - acc: 0.8091 - val_loss: 0.2110 - val_acc: 0.9639
Epoch 3/10
132/132 [==============================] - ETA: 0s - loss: 0.3281 - acc: 0.8806

132/132 [==============================] - 179s 1s/step - loss: 0.3281 - acc: 0.8806 - val_loss: 0.0991 - val_acc: 0.9691
Epoch 4/10
132/132 [==============================] - 182s 1s/step - loss: 0.2946 - acc: 0.8837 - val_loss: 0.0995 - val_acc: 0.9536
Epoch 5/10
132/132 [==============================] - ETA: 0s - loss: 0.2568 - acc: 0.9027

132/132 [==============================] - 177s 1s/step - loss: 0.2568 - acc: 0.9027 - val_loss: 0.0693 - val_acc: 0.9742
Epoch 6/10
132/132 [==============================] - ETA: 0s - loss: 0.2003 - acc: 0.9270

132/132 [==============================] - 179s 1s/step - loss: 0.2003 - acc: 0.9270 - val_loss: 0.0463 - val_acc: 0.9794
Epoch 7/10
132/132 [==============================] - 194s 1s/step - loss: 0.2111 - acc: 0.9240 - val_loss: 0.0571 - val_acc: 0.9845
Epoch 8/10
132/132 [==============================] - 176s 1s/step - loss: 0.2037 - acc: 0.9262 - val_loss: 0.0503 - val_acc: 0.9794
Epoch 9/10
132/132 [==============================] - 180s 1s/step - loss: 0.1925 - acc: 0.9300 - val_loss: 0.0631 - val_acc: 0.9691
Epoch 10/10
132/132 [==============================] - ETA: 0s - loss: 0.1805 - acc: 0.9308

132/132 [==============================] - 187s 1s/step - loss: 0.1805 - acc: 0.9308 - val_loss: 0.0430 - val_acc: 0.9948


Now, we save model

In [11]:
model.save('model.h5',history)